# Importing Libraries

In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

Scraping all Data from the website

In [2]:
# Website URL
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# Getting data from URL with Request Library
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

Getting the header for the table

In [3]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


Getting the table content

In [4]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table because we have 3 columns
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Finally transforming dictionary to dataframe for further process

In [5]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
cols = ['Postcode','Borough','Neighbourhood\n']
df = df.loc[:, cols]
df.rename(columns = {'Neighbourhood\n': 'Neighbourhood'},inplace = True) 
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


# Cleaning Data
Remove '\n' endings from Neighbourhood column

In [6]:
df['Neighbourhood'] = df['Neighbourhood'].map(lambda x: x.rstrip('\n'))

In [7]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


To easily drop missing values, replace 'Not assigned' with numpy nan values

In [8]:
df[df == 'Not assigned'] = np.nan

In [9]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [10]:
df.shape

(288, 3)

In [11]:
df = df.loc[df['Borough'].notna(),:]

In [12]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Now, we dropped missing values in the 'Borough' column. And we have 211 rows

In [13]:
df.reset_index(drop=True, inplace=True) # resetting index values

In [14]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


To easily merging strings according to Postcode, I implemented groupby method to dataframe and created a new dataframe as 'df_count'

In [15]:
df_count = df.groupby('Postcode').count()
df_count.reset_index(inplace=True)

In [16]:
df_count.head()

,Postcode,Borough,Neighbourhood
0,M1B,2,2
1,M1C,3,3
2,M1E,3,3
3,M1G,1,1
4,M1H,1,1


Before merging rows in Neigbourhood column, I looked for missing values in Neighbourhood column

In [17]:
df[df['Neighbourhood'].isna()]

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,NaN


Yes we have one missing value in 6th row. According to instructions I replaced missing value with its Borough

In [18]:
df.loc[6]['Neighbourhood'] = df.loc[6]['Borough']
df.head(7)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park


Now, we don't have any missing value in Neigbhourhood column

In [19]:
df[df['Neighbourhood'].isna()]

,Postcode,Borough,Neighbourhood


To merge neighbourhoods with same postcode, I created a new dataframe as 'df2' with same columns

In [20]:
df2 = pd.DataFrame(columns= df.columns)
df2

,Postcode,Borough,Neighbourhood


I find the neighbourhoods with the same postcode, then put them in df2 dataframe

In [21]:
for index in df_count['Postcode']:
    b = ''
    for a in df[df['Postcode'] == index]['Neighbourhood']:
        b += a+', '
    b = b.rstrip(', ')
    df3 = pd.DataFrame([[index,df[df['Postcode'] == index]['Borough'].iloc[0],b]],columns = df2.columns)
    df2 = df2.append(df3,ignore_index=True)

In [22]:
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [23]:
df2.shape

(103, 3)

# Geographical data is gathered from csv file

In [24]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Checking every element wheter they have same postcodes

In [25]:
np.sum(df2['Postcode']==geo_df['Postal Code'])

103

Yes we have same elements for two dataframes

In [26]:
geo_df.drop(columns = 'Postal Code', inplace = True) ## Dropping 'Postal Code' column because we don t need it after merging

In [27]:
new_df = pd.concat([df2,geo_df],axis=1)

Our new dataframe is ready below with location data.

In [28]:
new_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [29]:
new_df.shape

(103, 5)

# Toronto Data Analysis
New data frame is created to store Toronto data.

In [30]:
new_df2 = pd.DataFrame(columns= new_df.columns)
new_df2

,Postcode,Borough,Neighbourhood,Latitude,Longitude


We created a list only boroughs that contain the word Toronto.

In [31]:
list_t = []
for a in new_df['Borough']:
    if 'Toronto' in a:
        if a not in list_t:
            list_t.append(a)
list_t

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [32]:
new_df_2 = pd.DataFrame([],columns = new_df.columns)
for a in list_t:
    df = new_df[new_df['Borough']==a] 
    new_df_2 = new_df_2.append(df)

Now Our dataframe is ready.

In [33]:
new_df_2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
